In [5]:
import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import wandb

In [6]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%
100.0%
100.0%
100.0%


In [7]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [8]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [5]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return loss.item()

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return (correct, test_loss)

In [9]:
from datetime import datetime

datetime.now().isoformat()

'2025-11-26T15:35:12.737228'

In [ ]:
project="test-project"
config = {
    "learning_rate": 0.02,
    "architecture": "MLP",
    "dataset": "FashionMNIST",
    "epochs": 10,
}

myname = "myrun-"+datetime.now().isoformat()
with wandb.init(project=project, config=config, name=myname) as run:
  # This block simulates a training loop logging metrics
  epochs = config["epochs"]
  for epoch in range(1, epochs+1):
      print(f"Epoch {epoch}\n-------------------------------")
      train(train_dataloader, model, loss_fn, optimizer)
      acc,loss= test(test_dataloader, model, loss_fn)
      #Log metrics from your script to W&B
      run.log({"acc": acc, "loss": loss}) 
  print("Done!")

Epoch 1
-------------------------------
loss: 2.293036  [   64/60000]
loss: 2.283704  [ 6464/60000]
loss: 2.261383  [12864/60000]
loss: 2.257717  [19264/60000]
loss: 2.246151  [25664/60000]
loss: 2.201491  [32064/60000]
loss: 2.220376  [38464/60000]
loss: 2.178285  [44864/60000]
loss: 2.176144  [51264/60000]
loss: 2.132190  [57664/60000]
Test Error: 
 Accuracy: 40.4%, Avg loss: 2.138285 

Epoch 2
-------------------------------
loss: 2.152390  [   64/60000]
loss: 2.140655  [ 6464/60000]
loss: 2.079386  [12864/60000]
loss: 2.086570  [19264/60000]


In [42]:
torch.save(model.state_dict(), "test-model.pth")
print("Saved PyTorch Model State to test-model.pth")

Saved PyTorch Model State to test-model.pth


In [43]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("test-model.pth", weights_only=True))

<All keys matched successfully>

In [44]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
